# Project Summary
#### Devin Shirley, Madison Karlin, Katherine Koczwara, Melissa Stephens


### Part 1
Blasted 'uniquetranscripts.fasta' in the 'BLAST_DNA_hits' repo with the following parameters: 
* nucleotide collection
* normal defaults
* highly similar sequence option

Saved and downloaded hits tables to repo, see 'HitTable1',1-6

### Part 2
Using unix we made a table that includes the top hit for each transcript (6) 
* Chose with highest percent identity and highest max score

##### pseudo unix code
for files in 'all of the csvs'; get the top line of the document since it is sorted that way as a default, read just that line, move that to a file; repeat for each csv 

the script for this is 'top_hits.sh' saved in the 'BLAST_DNA_hits' repo


In [ ]:
#####Reads all of the .csv files in the repository, takes their first line
#and moves them to a new file

for file in *.csv; do head -1 $file | cat top_hits; done

##### Extra Notes
The disovered accession number from the table was searched on its NCBI page we then found the protein under the CDS subheading which is hyperlinked, after clicking on it we could see the name of the protein which was used in part 3

### Part 3
We searched the NCBI protein database (https://www.ncbi.nlm.nih.gov/protein) for protein from each top hit   
* filtered by taxon
    + View as a tree
* Choose 10-20 protein sequences from closely related species (Muridae is family mice/rats are in)
* Create 1 fasta file per transcript (6)
Added each file to the 'BLAST_Proteins' repo
    see Protein_1_Gluthanione....
    Protein_2....
    ....

While doing this step we also did the further exploration steps for nonmammalls (fish,turtles,birds, etc) and primates (humans, apes) and added their protein fasta files to the repo

For each group we saved the protein files to a repo inside the 'BLAST_Proteins' repo
* Mice/Rodents/Muridae to the 'Rodent_Protein_Hits' repo
* Humans/Apes to the 'Primate_Protein_Hits' repo
* Birds/Lizards/Turtles/Fish to the 'Nonmammalian_Protein_Hits' repo

### Part 4
We wrote a python script that translates RNAseq data to amino acid data, see 'aminoacid_seq.py' in the 'RNAseq_data' repo
* input .fasta file 
* output .protein file (same as fasta but with different extension) 
* list of amino acids for each transcript

##### pseudo python code

#import packages
#open infiles to read
#open codon map to read
#open outfiles to write
#read codonmap into a dictionary
#define the function to translate the base sequence column into the amino acid column
#define the function to group base sequence into codons
#define the function to translate the codons using the dictionary
    #for:
        #if the amino acid is Stop, stop reading the sequence
        #else keep translating
    #return the amino acid string
#define the start codon regex
#for:
    #if line begins with >
        #write line to the outfile
    #else
        #find start codon using regex and delete everything prior to that codon
        #use the codon translating function
        #write line to outfile
#repeat for loop in each each infile and write to corresponding outfiles
#close all infiles, outfiles, and the codonmap
        




In [ ]:
##actual python code 

#load packages
import numpy
import pandas
import re
from sys import argv
from itertools import takewhile

#openfiles
infile1 = open("control1.fasta","r")
infile2 = open("control2.fasta","r")
infile3 = open("obese1.fasta","r")
infile4 = open("obese2.fasta","r")
codonmap = open("codonmap.txt","r")

#create and open files for output
outfile1 = open("control1_protein","w")
outfile2 = open("control2_protein","w")
outfile3 = open("obese1_protein","w")
outfile4 = open("obese2_protein","w")

#create dictionary
d = {}
for line in codonmap:
    (val,key) = line.split()
    d[str(key)] = val

def translateCodon (x):
    if x in d:
        return d[x]

def splitCodons(sequence):
    result = []
    for i in range(1, len(sequence)+1, 3):
        result.append(sequence[i:i+3])
    return result
    
def translateSequence(sequence):
    codons = splitCodons(sequence)
    aaString = 'M'
    for codon in codons:
        if translateCodon(codon) == 'Stop':
            return aaString
        else:
            aaString += translateCodon(codon)
    return aaString

r1=r"([ATCG]{3,3})+?(ATG)"

for line in infile1:
	line = line.strip()
	if line[0] == ">":
	    outfile1.write(line + "\n")
	else:
	    start = re.sub(r1,"ATG",line,count=1)
	    aminoacids = translateSequence(start)
        outfile1.write(aminoacids + "\n")

for line in infile2:
	line = line.strip()
	if line[0] == ">":
	    outfile2.write(line + "\n")
	else:
	    start = re.sub(r1,"ATG",line,count=1)
	    aminoacids = translateSequence(start)
        outfile2.write(aminoacids + "\n")

for line in infile3:
	line = line.strip()
	if line[0] == ">":
	    outfile3.write(line + "\n")
	else:
	    start = re.sub(r1,"ATG",line,count=1)
	    aminoacids = translateSequence(start)
        outfile3.write(aminoacids + "\n")

for line in infile4:
	line = line.strip()
	if line[0] == ">":
	    outfile4.write(line + "\n")
	else:
	    start = re.sub(r1,"ATG",line,count=1)
	    #short = re.sub(r'.*M','M',start)
	    aminoacids = translateSequence(start)
        outfile4.write(aminoacids + "\n")
	
#close files 
infile1.close()
infile2.close()
infile3.close()
infile4.close()
outfile1.close()
outfile2.close()
outfile3.close()
outfile4.close()
codonmap.close()


##### Extra Notes
this code translated the RNAseq fasta files:
* 'contro1.fasta'
* 'control2.fasta'
* 'obese1.fasta'
* 'obese2.fasta'
into amino acid sequences and saved the files as:
* 'control1.protein'
* 'control2.protein'
* 'obese1.protein'
* 'obese2.protein'
all of these files, along with the codon map used for the dicitonary and the python script used to accomplish the task are locate in the 'RNAseq_data' file 

### Part 5
Built a HMM of the 6 transcript proteins, search translated RNAseq files 
* Muscle alignment --> hmmbuild --> hmmsearch
* use bash script to loop over files

In [ ]:
#Psuedo code

#use muscle to create alignments
muscle -in <inputfule> -out <outputfile> #input file is the protein fasta file from NCBI
    #repeat for each protein (6) for loop here
    
#use hmm build to build the markov model
hmmbuild <hmm_file_out> <MSAfile> #MSAfile is the output of the muscle alignment, hmm_file_out builds it
    #repeat for each protein (6) for loop here too

#loop through each model, loop through each treatment, concatenate to build one table for each treatment
for files in proteinmodels
do
    for treatment in 'translated RNAseq files'
    do    
        hmmsearch --tblout $files.$hmms.hits $files $hmms #searches first protein, repeats for each protein and treatment
        cat $files.$hmms.hits >> $files.hits.table #adds protein hits to new files for each treatment, appends 
    done
done

#for these steps we need the protein fasta files, the converted RNAseq files, the bioinformatic tools, 
#and the necessary script in the appropriate repository


In [ ]:
#builds hidden markov models for each protein and searches translated RNAseq files for each protein
#have to have muscle, hmmbuild, and hmmmsearch files in repo

#muscle alignment
for proteins in protein*.fasta #loops through all protein files
do 
	./muscle -in $proteins -out $proteins.align
done
#build models
for align in protein*.fasta.align #builds hmms for each protein alignment
do
	./hmmbuild $align.hmm $align
done
#search rnaseq files for protein models
for models in protein*.fasta.align.hmm #loops through models
do
	for treatment in *.protein #Loops through treatments
	do
		./hmmsearch --tblout $treatment.hits $models $treatment #creates table for each transcript
		cat $treatment.hits >> $treatment.hits.table #appends to table
	done
done



##### Extra Notes
For this code to work we have 3 repos for each time we wanted to build and search the protein models
These are located in the following repositories:
* 'Rodent_HMMs'
* '
*


### Part 6
We Graphically compared expression levels across groups
* Qualitatively compare results to Kuhns & Pluznick 2017

In [ ]:
#pseudo code

# cat all hits files for each transcript
# generate a table of counts - transcript, model, hits
# read table into dataframe
# 4 graphs - subset data (one for each sample, with 6 models)

In [ ]:
#actual unix code (see script Comparions.sh) in the 'Plots' repo

grep -v '#' control1.protein.hits.table_rodent | awk '{print $1,$3}' | uniq -c | awk '{print $1,$2,$3}' > Control1Counts.txt
grep -v '#' control2.protein.hits.table_rodent | awk '{print $1,$3}' | uniq -c | awk '{print $1,$2,$3}' > Control2Counts.txt
grep -v '#' obese1.protein.hits.table_rodent | awk '{print $1,$3}' | uniq -c | awk '{print $1,$2,$3}' > Obese1Counts.txt
grep -v '#' obese2.protein.hits.table_rodent | awk '{print $1,$3}' | uniq -c | awk '{print $1,$2,$3}' > Obese2Counts.txt

In [ ]:
#actual python code (see script ComparisonGraphs.py) in the 'Plots' repo

import pandas
from plotnine import *

C1data=pandas.read_table("Control1Counts.txt")
C2data=pandas.read_table("Control2Counts.txt")
O1data=pandas.read_table("Obese1Counts.txt")
O2data=pandas.read_table("Obese2Counts.txt")

C1data.columns=['Count','sample','Model']
C2data.columns=['Count','sample','Model']
O1data.columns=['Count','sample','Model']
O2data.columns=['Count','sample','Model']

ggplot(C1data,aes(x='Model',y='Count'))+geom_col(size=20)+ggtitle('Control1 Transcript Counts')+ theme ( axis_text_x = element_text ( angle = 90 ))
ggplot(C2data,aes(x='Model',y='Count'))+geom_col(size=20)+ggtitle('Control2 Transcript Counts')+ theme ( axis_text_x = element_text ( angle = 90 ))
ggplot(O1data,aes(x='Model',y='Count'))+geom_col(size=20)+ggtitle('Obese1 Transcript Counts')+ theme ( axis_text_x = element_text ( angle = 90 ))
ggplot(O2data,aes(x='Model',y='Count'))+geom_col(size=20)+ggtitle('Obese2 Transcript Counts')+ theme ( axis_text_x = element_text ( angle = 90 ))

This generates 4 histograms (one for each sample type) with the 6 protein models. These can be seen in the "plots" repository as C1CountPlot.png, C2CountPlot.png, O1CountPlot.png, O2CountPlot.png

In [ ]:
#### Manual test

#Using the following unix code and excel, we also generated a csv file containing information regarding the frequency of hits
#see the 'expression.script' in the 'Plots' repository:
cat control1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein1'
cat control1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein2'
cat control1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein3'
cat control1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein4'
cat control1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein5'
cat control1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein6'
cat control2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein1'
cat control2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein2'
cat control2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein3'
cat control2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein4'
cat control2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein5'
cat control2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein6'
cat obese1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein1'
cat obese1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein2'
cat obese1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein3'
cat obese1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein4'
cat obese1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein5'
cat obese1.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein6'
cat obese2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein1'
cat obese2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein2'
cat obese2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein3'
cat obese2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein4'
cat obese2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein5'
cat obese2.protein.hits.table_nonmammalian | grep -v '#' | grep -c 'protein6'


#after generating the table, we then plotted the table in python using the 'quickdirtyplot.py' located in the 'Plots' directory

In [ ]:
import pandas
from plotnine import *
data=pandas.read_csv("expressionlevels.csv")
plot=ggplot(data,aes(x='Protein',y='ExpressionLevel',color='Treatment'))
plot+geom_jitter(width=.1,height=.3,alpha=.5,size=5)+theme_classic()+ggtitle('Protein Expression Levels for Each Treatment')


This generates the graph seen in the image 'expressionlevels.png' in the 'Plots' repository 


6 models are abbreviated Lhx2,Gsta2,Slc7a12,Ptpn5,Atp12a,Synpr

#### Qualitative comparison to Kuhns and Ploznick 2017
Our results mirror those of Kuhns and Ploznick. Both Obese samples show highest fold change in Synpr, followed by Lhx2, Atp12a, and Ptpn5. Greatest fold decrease is seen in genes Slc7a12 and Gsta2

### Further Exploration
#### 1. Change the 'Optimize for' option
##### Qualitatively, how do discontinuous megablast (more disimilar) and blastn (somewhat similar) change your table of BLAST hits?
The top hits for each of these blasts tend to remain the same. However, when you make the blast more disimilar (discontinous megablast) or only somewhat similar (blastn) the range of results changes. You get more sequences matching to other species and the quality of matches (e value of matches with lower max scores) also decreases. 
#### 2. Explore the effects of phylogenetic relatedness of your amino acid sequences on the performance of your HMM model
##### What happens if you build your HMM protein model using more distantly related mammals (e.g. primates)?
##### Do you still get the same quality hits if your HMM protein model is based on non-mammalian sequences?
Generally, you can make some assumptions that E-value scores tend to increase as phylogenetic relationship decreases. Looking at the quality hits of other HMM protein models (between nonmammalian, rodents, and primates) shows that E-values are typically lower in nonmammals than in the mammals (rodents and primates). However, this was not always the case when you compared the rodent hmms to the primate hmms with the E-values increasing for one protein or decreasing for others. Therefore, for more closely related species you may get different e-value scores (quality hits) for each protein. 